In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import datetime

# from bokeh.io import output_file, show
# from bokeh.models import ColumnDataSource, GMapOptions
# from bokeh.plotting import gmap

from PIL import Image, ImageDraw
import os
import shutil

from mpl_toolkits.basemap import Basemap

%matplotlib inline

#Turn off interactive plotting for pyplot
plt.ioff()

In [2]:
df = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_Reduced_Trackfile.txt',header=None,names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"],low_memory=False,sep='\t')
df = df.drop("Unused",axis=1)
ln = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_WWLLN_Locations.txt',header=None,names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"],low_memory=False,sep=' ')

minLong_, minLat_, maxLong_, maxLat_ = ln['Long'].min(), ln['Lat'].min(), ln['Long'].max(), ln['Lat'].max()
buffer = 10

In [3]:
# Create interpolated data
dfc = df.copy()
for i in range(60):
    s = dfc.iloc[i]
    s2 = dfc.iloc[i+1]
    newLat = np.linspace(s.Lat,s2.Lat,num=6)
    newLong = np.linspace(s.Long,s2.Long,num=6)
    newMin = np.linspace(s.Min_Pressure,s2.Min_Pressure,num=6)
    newWinds = np.linspace(s.Max_Winds, s2.Max_Winds, num=6)
    newHours = np.arange(s.Hour,s.Hour + 6)
    newYears = [s.Year for x in range(6)]
    newMonths = [s.Month for x in range(6)]
    newDays = [s.Day for x in range(6)]
    newDf = pd.DataFrame([newYears, newMonths, newDays, newHours, newLat, newLong, newMin, newWinds]).T
    newDf.columns=['Year','Month','Day','Hour','Lat','Long','Min_Pressure','Max_Winds']
    dfc = dfc.append(newDf,ignore_index=True)

In [74]:
df, dfc = dfc, df # Swap the two dataframes
#Graph the center of the storm and lightning
for x in range(len(df['Month'].unique())):
    month = df['Month'].unique()[x]
    for y in range(len(df[df['Month'] == df['Month'].unique()[x]]['Day'].unique())):
        day = df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]
        for z in range(len(df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique())):
            hour = df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique()[z]
            for w in range(0,60,30):
                minute = w
                #The above gets the month, day, hour, minute (before 30 minutes or after 30 minutes)
                #Make big map
                fig = plt.figure(figsize=(30,15))
                m = Basemap(llcrnrlon=minLong_-buffer, llcrnrlat=minLat_-buffer,urcrnrlon=maxLong_+buffer,urcrnrlat=maxLat_+buffer,lon_0=0,lat_0=0)
                m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
                m.fillcontinents(color='grey', alpha=0.7, lake_color='grey')
                m.drawcoastlines(linewidth=0.1, color="white")
                #Plot ticks for lat/long
                plt.xticks(np.arange(minLong_-buffer,maxLong_+buffer,step=5))
                plt.yticks(np.arange(minLat_-buffer,maxLat_+buffer,step=5))
                #Title the map 2017:month:day::hour:minute
                plt.title("2017:" + '{:2d}'.format(int(month)) + ":" + '{:2d}'.format(int(day)) + "::" + '{:2d}'.format(int(hour)) + ":" + '{:2d}'.format(int(minute)))
                #Plot the center of Irma
                m.plot(df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour)]['Long'], df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour)]['Lat'], linestyle='none', marker="o", markersize=30, alpha=1, c="red", markeredgecolor="black", markeredgewidth=1)
                #Plot all of the lightning that appears on that month/day/hour/minute section
                m.plot(ln[(ln['Month'] == month) & (ln['Day'] == day) & (ln['Hour'] == hour) & (ln['Min'] >= minute) & (ln['Min'] <= minute + 30)]['Long'],ln[(ln['Month'] == month) & (ln['Day'] == day) & (ln['Hour'] == hour) & (ln['Min'] >= minute) & (ln['Min'] <= minute + 30)]['Lat'], linestyle='none', marker="X", markersize=8, alpha=.7, c="yellow", markeredgecolor="black", markeredgewidth=1)
                #Save and close the figure
                plt.savefig("./data/Irma/" + "2017_" + str(int(month)) + "_" + str(int(day)) + "_" + str(int(hour)) + "_" + str(int(minute)),bbox_inches='tight')
                plt.close(fig)

In [4]:
dfc

,Year,Month,Day,Hour,Lat,Long,Min_Pressure,Max_Winds
0,2017.0,8.0,27.0,18.0,11.50,-16.00,0.0,25.0
1,2017.0,8.0,28.0,0.0,11.50,-17.80,0.0,25.0
2,2017.0,8.0,28.0,6.0,11.60,-19.30,1009.0,25.0
3,2017.0,8.0,28.0,12.0,11.90,-20.50,1009.0,25.0
4,2017.0,8.0,28.0,18.0,12.30,-21.00,1008.0,25.0
...,...,...,...,...,...,...,...,...
416,2017.0,9.0,11.0,13.0,29.86,-82.86,972.0,57.0
417,2017.0,9.0,11.0,14.0,30.12,-83.02,974.0,54.0
418,2017.0,9.0,11.0,15.0,30.38,-83.18,976.0,51.0
419,2017.0,9.0,11.0,16.0,30.64,-83.34,978.0,48.0
